In [1]:
import bcrypt

def verify_password(plain_password: str, hashed_password: str) -> bool:
    password_byte_enc = plain_password.encode('utf-8')
    return bcrypt.checkpw(password = password_byte_enc , hashed_password = hashed_password)

def get_password_hash(password: str) -> str:
    pwd_bytes = password.encode('utf-8')
    salt = bcrypt.gensalt()
    hashed_password = bcrypt.hashpw(password=pwd_bytes, salt=salt)
    return hashed_password

In [2]:
get_password_hash("hej")

b'$2b$12$.foqoWpkl/eFGrTPMjcNtOGJ6ZJYnToG7AlU63JlPQq9dFAhMSmR2'

In [4]:
get_password_hash("hej")+get_password_hash("alex")

b'$2b$12$HObp0vHPGxE7K5A0At4GMumheS5KdZRV95xhbScahnSrL5MA3gBaK$2b$12$vLPdq.OTTYomRUrLLAS3euhjwoQRGVJiFl7LCVXx0/UAqhgmdT/3O'

In [1]:
import base64
import secrets
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

KDF_ALGORITHM = hashes.SHA256()
KDF_LENGTH = 32
KDF_ITERATIONS = 120000

def encrypt(plaintext: str, password: str) -> (bytes, bytes):
    # Derive a symmetric key using the passsword and a fresh random salt.
    salt = secrets.token_bytes(16)
    kdf = PBKDF2HMAC(
        algorithm=KDF_ALGORITHM, length=KDF_LENGTH, salt=salt,
        iterations=KDF_ITERATIONS)
    key = kdf.derive(password.encode("utf-8"))

    # Encrypt the message.
    f = Fernet(base64.urlsafe_b64encode(key))
    ciphertext = f.encrypt(plaintext.encode("utf-8"))

    return ciphertext, salt

def decrypt(ciphertext: bytes, password: str, salt: bytes) -> str:
    # Derive the symmetric key using the password and provided salt.
    kdf = PBKDF2HMAC(
        algorithm=KDF_ALGORITHM, length=KDF_LENGTH, salt=salt,
        iterations=KDF_ITERATIONS)
    key = kdf.derive(password.encode("utf-8"))

    # Decrypt the message
    f = Fernet(base64.urlsafe_b64encode(key))
    plaintext = f.decrypt(ciphertext)

    return plaintext.decode("utf-8")

def main():
    password = "aStrongPassword"
    message = "a secret message"

    encrypted, salt = encrypt(message, password)
    decrypted = decrypt(encrypted, password, salt)

    print(f"message: {message}")
    print(f"encrypted: {encrypted}")
    print(f"decrypted: {decrypted}")

In [2]:
encrypt('mittlössenord', 'aloha')

(b'gAAAAABl4n0uHt7JfZksd-bPo1i8AraT-8-USQX2ROBpPvlOZMBNwLFPPLUjNkVq1w322eADNUPkdp3TZBqdrFYee0IxhBu3cA==',
 b'\xbe\xd7\x01\xbb\xdc\\\xb6e\x1b\t6\xc0C\xe2v\x17')

In [3]:
password = "aStrongPassword"
message = "a secret message"

encrypted, salt = encrypt(message, password)
decrypted = decrypt(encrypted, password, salt)

print(f"message: {message}")
print(f"encrypted: {encrypted}")
print(f"decrypted: {decrypted}")

message: a secret message
encrypted: b'gAAAAABl4n1UJc9_3mJamJgGR-AdPpnRxiDgu6j13rEaCuIsd9vogAukSouyd_UDrVT7lLi-OCMYvG_xvB6vhTb9qSFKukLcCIdUN-7Hwrlxzqs3gp2bzk0='
decrypted: a secret message
